# ILYAS ABOU EL MAJD

---

# Rapport d'Erreurs et de Solutions - Gestionnaire de Mots de Passe avec Chiffrement RSA et César

## Introduction
Ce rapport présente les différentes erreurs rencontrées lors de l'implémentation d'un gestionnaire de mots de passe utilisant le chiffrement RSA et César. Il met en lumière les problèmes spécifiques identifiés dans plusieurs fonctions clés du programme, ainsi que les solutions apportées pour garantir le bon fonctionnement du code.

---

## 1. Problème : Génération de Mots de Passe (generer_mot_de_passe)
**Erreur Observée :**  
Lors de la génération de mots de passe, les mots du dictionnaire étaient mal slicés, ce qui rendait les mots de passe incomplets et sans signification.

**Cause :**  
Absence de conditions pour filtrer correctement les mots utilisés.

**Solution Apportée :**  
Ajout de conditions supplémentaires pour s'assurer que les mots possibles respectent des critères précis (ex : longueur minimale/maximale). Cela a permis d'obtenir des mots de passe valides et conformes.

---

## 2. Problème : Génération de Mots de Passe (generer_mot_de_passe)
**Erreur Observée :**
Initialement, il était difficile de définir une métrique de robustesse pour le générateur de mots de passe. La longueur du mot de passe et la robustesse souhaitée semblaient indépendantes, ce qui créait une confusion dans la logique.

**Cause :**
La métrique de robustesse n'était pas clairement définie et la relation entre la longueur du mot de passe et le niveau de robustesse n'était pas évidente.

**Solution Apportée :**
J'ai redéfini la robustesse en fonction du nombre précis de caractères. Chaque niveau de robustesse (faible, moyenne, forte) est désormais associé à une plage spécifique de longueur de mot de passe. Cela permet de garantir que la robustesse et la longueur sont liées et cohérentes.

---

## 3. Problème : Recherche et Affichage des Lignes (rechercher_afficher_lignes)
**Erreur Observée :**  
Lors de la recherche dans les données de connexion, les mots de passe chiffrés étaient mal interprétés, provoquant des erreurs au moment de leur déchiffrement.

**Cause :**  
Absence de transformation correcte de la chaîne chiffrée en liste utilisable par la fonction de déchiffrement RSA.

**Solution Apportée :**  
Implémentation de la ligne suivante :
```python
mot_de_passe_decrypte = décrypter_password(eval(mot_de_passe.replace(" (RSA)", "")), private_key)
```
Cette modification permet de transformer correctement le mot de passe crypté en une liste exploitable par la fonction de déchiffrement.

---

## 4. Problème : Restauration des Mots de Passe (restaurer_mdp)
**Erreur Observée :**  
Le pattern regex utilisé pour rechercher les mots de passe arrêtait l'extraction à la première virgule, ce qui empêchait de déchiffrer correctement les mots de passe RSA.

**Cause :**  
Pattern regex inadapté, limité par la présence de caractères spécifiques dans le mot de passe.

**Solution Apportée :**  
Utilisation du pattern suivant :
```python
match = re.search(r"Password:\s*(\[[^\]]+\]|[^\s,]+)", ligne)
```
Ce pattern permet de capturer les mots de passe, même s'ils contiennent des virgules ou d'autres caractères.

---

## 5. Problème : Modification/Ajout de Mots de Passe (add_mod_mdp)
**Erreur Observée :**  
Lors de la modification des fichiers de mots de passe, l'absence de la commande `truncate()` provoquait un excédent de texte, rendant le fichier peu lisible.

**Cause :**  
Sans la commande `truncate()`, le fichier conserve des parties inutiles lorsqu'il est réécrit.

**Solution Apportée :**  
Ajout de la ligne suivante après l'écriture du fichier :
```python
fichier.truncate()
```
Cela garantit que le fichier est correctement nettoyé après modification.

---

## 6. Problème : Restauration des Anciennes Versions des Mots de Passe (restaurer_mdp)
**Erreur Observée :**  
Difficulté à restaurer efficacement les versions précédentes des mots de passe.

**Solution Apportée :**  
- Implémentation d'une méthode basée sur la date.
- Copie de la date et recherche de cette date dans `history.txt` à l'aide d'un pattern regex. Cela permet de retrouver précisément les anciennes versions.

---

## 7. Problème : Protection des Clés RSA
**Erreur Observée :**  
Difficulté à protéger efficacement les clés publiques et privées.

**Solution Apportée :**  
- La clé publique est stockée en clair.
- La clé privée est chiffrée à l'aide du chiffrement de César avant d'être enregistrée dans `key.txt`.
- Le fichier `key.txt` est créé uniquement si aucun fichier n'existe dans le dossier.

Code associé :
```python
if not os.path.exists("key.txt"):
    public_key, private_key = générer_cles()
    with open("key.txt", "w") as fichier:
        fichier.write(f"Public Key: {public_key}, Private Key: {crypter_key(private_key)} (CRYPTE)")
```
Cela garantit une meilleure sécurisation des clés sensibles.

---

## Conclusion
L'implémentation de ce gestionnaire de mots de passe a permis d'acquérir une meilleure compréhension des concepts de chiffrement (RSA et César) et des techniques de manipulation de fichiers en Python. Les erreurs rencontrées ont contribué à l'amélioration continue du projet et à l'apprentissage de bonnes pratiques de développement.

---

# Choix algorithmiques pour le gestionnaire de mots de passe

## Génération d'un nombre premier

La fonction `générer_prime` permet de générer un nombre premier aléatoire dans un intervalle donné. Ce choix garantit que les nombres premiers générés sont suffisamment grands et aléatoires, tout en assurant une efficacité acceptable.



## Calcul du Plus Grand Commun Diviseur (PGCD)

La fonction `pgcd` utilise l'algorithme d'Euclide pour déterminer le PGCD entre deux nombres. Cet algorithme a été choisi pour sa simplicité et son efficacité, même avec des nombres de grande taille, ce qui est crucial pour les calculs impliqués dans l'algorithme RSA.



## Génération des clés RSA

La fonction `générer_cles` génère les paires de clés publique et privée nécessaires au chiffrement RSA. Les choix algorithmiques incluent :

- La génération de deux grands nombres premiers `p` et `q`, qui assurent une forte sécurité pour le chiffrement.
- Le calcul de `phi = (p-1)*(q-1)` pour définir l'espace des clés possibles.



## Génération d'un mot de passe sécurisé

La fonction `generer_mot_de_passe` permet de générer des mots de passe aléatoires répondant à des critères de sécurité. Les choix algorithmiques incluent :

- L'utilisation de bibliothèques standard comme `random` pour garantir l'unicité et l'imprévisibilité des mots de passe générés.
- La combinaison de caractères alphabétiques, numériques et spéciaux, ce qui rend les mots de passe plus difficiles à deviner.
- La flexibilité donnée à l'utilisateur pour choisir la longueur du mot de passe.



## Vérification d'un login existant

La fonction `verifier_login` parcourt le fichier contenant les identifiants pour vérifier si un login existe déjà. Les choix algorithmiques incluent :

- La lecture du fichier ligne par ligne pour éviter de charger tout le fichier en mémoire, ce qui est utile pour de grands ensembles de données.
- L'utilisation d'une condition logique pour retourner la ligne correspondante, facilitant ainsi l'accès aux informations associées à ce login.



## Ajout ou modification d'un mot de passe

La fonction `add_mod_mdp` gère l'ajout ou la modification d'un mot de passe. Les choix algorithmiques incluent :

- La vérification préalable de l'existence du login grâce à `verifier_login`, évitant ainsi les doublons.
- L'utilisation de logins.txt pour stocker les données, permettant une persistance des informations.
- L'enregistrement des métadonnées telles que le type de chiffrement et les labels, ce qui améliore la gestion et la recherche des mots de passe.



## Cryptage avec le code César

La fonction `cryptage_cesar` applique un décalage sur les caractères pour chiffrer un mot de passe. Les choix algorithmiques incluent :

- L'utilisation de l'arithmétique modulaire pour gérer les dépassements de la plage des caractères ASCII.
- Un décalage fixe pour simplifier l'implémentation, tout en offrant une protection de base contre l'accès non autorisé.



## Cryptage et décryptage des clés RSA

La fonction `cryptage_key` applique un chiffrement basique sur les clés privées avant leur stockage. Les choix algorithmiques incluent :

- Une méthode simple pour transformer les clés en chaînes cryptées, évitant ainsi leur exposition directe.
- L'utilisation d'une clé de chiffrement interne pour sécuriser les informations stockées.



## Suppression d'un login

La fonction `supprimer_login` permet de supprimer un login et ses informations associées. Les choix algorithmiques incluent :

- La création d'un fichier temporaire pour réécrire les données sans le login supprimé, ce qui garantit l'intégrité des données restantes.



## Restauration d'un mot de passe

La fonction `restaurer_mdp` permet de restaurer une ancienne version d'un mot de passe. Les choix algorithmiques incluent :

- La consultation de l'historique des mots de passe pour identifier la version souhaitée.
- La détection automatique du type de chiffrement utilisé (César ou RSA), permettant de restaurer correctement le mot de passe sans intervention supplémentaire de l'utilisateur.



## Recherche et affichage des lignes

La fonction `rechercher_afficher_lignes` facilite la recherche de données spécifiques dans le fichier des identifiants. Les choix algorithmiques incluent :

- La segmentation de la recherche en trois critères (login, label ou mot de passe), rendant la fonction plus flexible et intuitive.
- La lecture ligne par ligne avec des expressions régulières pour identifier les correspondances, offrant une recherche rapide et précise.
- La prise en compte du décryptage RSA lors de la recherche par mot de passe, ce qui garantit que les résultats correspondent aux données réelles, même chiffrées.



## Affichage des lignes choisies

La fonction `print_ligne_choisies` affiche les résultats de la recherche de manière claire et structurée. Les choix algorithmiques incluent :

- Un formatage cohérent des données pour améliorer la lisibilité et l'expérience utilisateur.
- Une validation supplémentaire des entrées pour s'assurer que les données affichées sont correctes et pertinentes.



## Conclusion

En résumé, chaque choix algorithmique a été guidé par les principes de sécurité, d'efficacité et de facilité d'utilisation, afin de créer un gestionnaire de mots de passe robuste et polyvalent.